In [7]:
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator

import numpy as np
import pandas as pd
import PyQt5
import requests
import urllib.request
import re
import os 

from datetime import datetime, timedelta
from time import gmtime, strftime
import numpy as np
import shutil
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [28]:
def genperate_data(staNum, df_input):   
    getMonth = df_input.index[0].month
    getYear  = df_input.index[0].year
    
    start   = dt.datetime(getYear, getMonth, 1) # Start from this date
    
    try:
        end     = dt.datetime(getYear, getMonth, 31)
    except:
        end     = dt.datetime(getYear, getMonth, 30)
        
    intval  = end -start
    dataNum = intval.days
    Counter = np.zeros(dataNum + 1)
    dayIndex = pd.date_range(start,end)
    df_Counter = pd.DataFrame({'TimeStamp': dayIndex, 'freq': Counter})
    df_Counter = df_Counter.set_index('TimeStamp')
    
#     The format of the txt file is
#      time = 20190601000745
#      time = 20190601001745
#      time = 20190601002745
#      time = 20190601003745
#      time = 20190601004745
#      time = 20190601005745
#      ....... etc.
    
#     fidName = "C:/Users/gtra389/Desktop/HL_" + staNum + ".txt"
#     fidName = "C:/Users/gtra389/Desktop/test.txt"
#     df_MeasSeries = pd.read_table(fidName, header=None, sep = "=")
#     df_MeasSeries.dropna(inplace = True)
#     df_MeasSeries = df_MeasSeries.drop(0,axis = 1)
#     df_MeasSeries.columns = ["Col1"]   
#     df_MeasSeries["Col1"] = df_MeasSeries.astype(str) 

#     df_MeasSeries["Col2"] = df_MeasSeries["Col1"].str[0:8] # Format: YYYYMMDD
#     df_MeasSeries['Time'] = df_MeasSeries['Col1'].str[8:14]# Format: hhmmss
#     df_MeasSeries.drop('Col1', axis=1, inplace=True)
#     df_MeasSeries = df_MeasSeries.set_index(pd.to_datetime(df_MeasSeries["Col2"],format = '%Y%m%d'))
#     df_MeasSeries.drop('Col2', axis=1, inplace=True)
#     df_MeasSeries.index.name = "Date"
    
    tempCounter = 0
    for jj in range (len(df_Counter)):    
        for ii in range(len(df_input)):
            if (df_input.index[ii] == df_Counter.index[jj]):
                tempCounter += 1
        df_Counter.freq[jj] = tempCounter
        tempCounter = 0

    data = df_Counter.freq    
    dates = [start + dt.timedelta(days=i) for i in range(dataNum + 1)]
    return dates, data

def calendar_array(dates, data):
    k, i, j = zip(*[d.isocalendar() for d in dates])
    if (max(i) - min(i)) == 51:
        i = [53 if x == 1 else x for x in i] # isocalendar() only has 53 weeks
        i = np.array(i) - min(i)
    else:    
        i = np.array(i) - min(i)
    #i = np.array(i) - min(i)
    j = np.array(j) - 1
    ni = max(i) + 1

    calendar = np.nan * np.zeros((ni, 7))
    calendar[i, j] = data
    
    return k, i, j, calendar
 
def calendar_heatmap(ax, dates, data, staNum):
    k, i, j, calendar = calendar_array(dates, data)
    cmapCus = matplotlib.colors.LinearSegmentedColormap.from_list("", ["white","red","yellow","green"])
    im = ax.imshow(calendar, interpolation='none', cmap= cmapCus)
    label_days(ax, dates, i, j, calendar)
    label_months(ax, dates, i, j, calendar)
    #ax.figure.colorbar(im)
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="10%", pad=0.05)
    plt.colorbar(im, cax=cax)
    im.set_clim(0, 1440)
    ax.set_title("XT {} (1440 samples / day)".format(staNum),y = 1.1, fontsize= 14)

def label_days(ax, dates, i, j, calendar):
    ni, nj = calendar.shape
    day_of_month = np.nan * np.zeros((ni, 7))
    day_of_month[i, j] = [d.day for d in dates]
 
    for (i, j), day in np.ndenumerate(day_of_month):
        if np.isfinite(day):
            ax.text(j, i, int(day), ha='center', va='center')
 
    ax.set(xticks=np.arange(7),
           xticklabels=['M', 'T', 'W', 'R', 'F', 'S', 'S'])
    ax.xaxis.tick_top() 
    
def label_months(ax, dates, i, j, calendar):
    month_labels = np.array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',
                             'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    months = np.array([d.month for d in dates])
    #months = np.array(["{}{}".format(d.year, d.month) for d in dates])
    
    uniq_months = sorted(set(months))
    yticks = [i[months == m].mean() for m in uniq_months]
    labels = [month_labels[m - 1] for m in uniq_months]
    ax.set(yticks=yticks)
    ax.set_yticklabels(labels, rotation=90)

def query_data_XT(arg1, ym):
    global flag
    #csv_data   = ''
    try: 
        flag = True
        # date = datetime.now().strftime("%Y%m")
        url = 'http://ec2-54-175-179-28.compute-1.amazonaws.com/get_csv_xitou.php?device_id='+str(arg1)+'&year_month='+str(ym)
        # print("url:" + url)
        # Get the URL of the csv file
        csv_LINK_PATTERN = 'href="(.*)">Download'
        req  = urllib.request.Request(url)
        html = urllib.request.urlopen(req)
        doc  = html.read().decode('utf8')

        string1 = "'>Download ID" + str(arg1) + str(ym) +" Data</a><br>"
        get_rul_patten = doc.strip(string1).strip("<a href='")
        file_name      = str(arg1) + "_csvFid"
        server_path    = "http://ec2-54-175-179-28.compute-1.amazonaws.com/"+ get_rul_patten

        # Creat the folder to save the csv file
        if not os.path.exists('./'+ file_name):
            os.makedirs('./'+ file_name)
        urllib.request.urlretrieve(server_path,'./'+file_name+'/'+file_name+'.csv')

        # Create a dataframe from the URL by data crawling
        local_csv_pos = './'+file_name+'/'+file_name+'.csv'

        csv_data = pd.read_csv(local_csv_pos,sep=", |,, | = |= ", header=None, index_col=False, engine='python')
        csv_data = csv_data[[3]]

        colName = ['time']
        csv_data.columns = colName # weather column (溫度、大氣壓力、濕度、風速、風向、雨量)
        csv_data["time"] = csv_data.astype(str) 
        csv_data["Col1"] = csv_data["time"].str[0:8] # Format: YYYYMMDD
        csv_data["Col2"] = csv_data["time"].str[8:14]# Format: hhmmss
        csv_data.drop("time", axis=1, inplace = True)
        csv_data = csv_data.set_index(pd.to_datetime(csv_data["Col1"],format = '%Y%m%d'))
        csv_data.drop('Col1', axis=1, inplace=True)
        csv_data.index.name = "Date"
        csv_data.columns = colName
        
        
        return csv_data
    
    except:
        print("No data {} received in {}".format(arg1, ym))
        flag = False
    

In [31]:
idNumDict  = [{'name':'鳳凰茶園','id':'4008'},
              {'name':'武岫農圃','id':'4005'},
              {'name':'溪頭活動','id':'4007'},
              {'name':'溪頭辦公','id':'4002'},
              {'name':'溪頭苗園','id':'4003'},
              {'name':'內湖國小','id':'4004'},
              {'name':'神    木','id':'4006'},
              {'name':'米堤飯店','id':'4010'},
              {'name':'天 文 台','id':'4011'},
              {'name':'杉林溪飯','id':'4012'},
              {'name':'竹山本部','id':'4001'},
              {'name':'下坪植物','id':'4009'}]

YYYYMM = ['201910','201911','201912']
for ii in range(0, len(idNumDict)):       
    staNum = idNumDict[ii]["id"]
    for yyyymm in YYYYMM:    
        df = query_data_XT(staNum, yyyymm)
        if flag:
            dates, data = genperate_data(staNum, df)
            fig, ax = plt.subplots(figsize=(6, 5.5))
            calendar_heatmap(ax, dates, data, staNum)
            plt.savefig('heatmap_calendar_{}_{}.jpg'.format(staNum, yyyymm), dpi=300)
            print("The heatmap calendar {} {} is done".format(staNum, yyyymm))
            plt.close()

The heatmap calendar 4008 201910 is done
The heatmap calendar 4008 201911 is done
The heatmap calendar 4008 201912 is done
The heatmap calendar 4005 201910 is done
The heatmap calendar 4005 201911 is done
The heatmap calendar 4005 201912 is done
The heatmap calendar 4007 201910 is done
The heatmap calendar 4007 201911 is done
The heatmap calendar 4007 201912 is done
The heatmap calendar 4002 201910 is done
The heatmap calendar 4002 201911 is done
The heatmap calendar 4002 201912 is done
The heatmap calendar 4003 201910 is done
The heatmap calendar 4003 201911 is done
The heatmap calendar 4003 201912 is done
No data 4004 received in 201910
The heatmap calendar 4004 201911 is done
No data 4004 received in 201912
The heatmap calendar 4006 201910 is done
The heatmap calendar 4006 201911 is done
The heatmap calendar 4006 201912 is done
The heatmap calendar 4010 201910 is done
The heatmap calendar 4010 201911 is done
The heatmap calendar 4010 201912 is done
The heatmap calendar 4011 201910 i